In [2]:
import numpy as np
import pandas as pd
import os
import re

from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Phrases
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import FastText


from keras.models import Sequential
import gensim
from gensim.models import FastText
from keras.layers import Embedding,Dense,LSTM, Dropout, Flatten, Convolution1D, MaxPooling1D


from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')
%matplotlib inline

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Fasttext Model

In [3]:
from gensim.models.wrappers import FastText
model = FastText.load_fasttext_format('wiki.en/wiki.en.bin')

### preprocessing data

In [4]:
#Preprocessing - removing unwanted characters, tokenization, stop-word removal
def clean_data(txts):
    x = re.sub("[^a-zA-Z]", " ",txts) 
    x = x.lower().split()                             
    stops = set(stopwords.words("english"))                  
    words = [w for w in x if not w in stops] 
    
    return( " ".join(words))

In [5]:
def clean_norm(txts):
    x = re.sub('(OntoBiotope |)','',txts)
    x = re.sub('(:|_)',' ',x)
    x = re.sub('(Taxonomy)','taxon',x)
    
    return x

### Train_norm+Dev_norm appended (same as Train_norm+ner, Dev_norm+ner appended)

In [7]:
data = pd.read_csv('data.csv')

data['norm2'].fillna(value='', inplace=True)
data['norm3'].fillna(value='', inplace=True)
data['norm4'].fillna(value='', inplace=True)
data['norm5'].fillna(value='', inplace=True)

data = data.drop(data[data.entity == 'Paragraph'].index)
data = data.drop(data[data.entity == 'Title'].index)
print(data.shape)
data.head()

(3268, 7)


,words,entity,norm1,norm2,norm3,norm4,norm5
0,selective broths based on hypertonic strontium...,Habitat,OntoBiotope OBT:000360,,,,
1,selective broths based on the bi-selenite ion,Habitat,OntoBiotope OBT:000360,,,,
2,Salmonellae,Microorganism,NCBI_Taxonomy 590,,,,
3,egg products,Habitat,OntoBiotope OBT:001086,,,,
4,egg,Habitat,OntoBiotope OBT:001847,,,,


In [8]:
data['words'] = data['words'].apply(lambda x: clean_data(x))

# data['norm1'] = data['norm1'].apply(lambda x: clean_norm(x))
# data['norm2'] = data['norm2'].apply(lambda x: clean_norm(x))
# data['norm3'] = data['norm3'].apply(lambda x: clean_norm(x))
# data['norm4'] = data['norm4'].apply(lambda x: clean_norm(x))
# data['norm5'] = data['norm5'].apply(lambda x: clean_norm(x))

In [9]:
train_words_ner = data['words'].tolist()
train_entity_ner = data['entity'].tolist()
#======================================================================
X_train_ner= np.zeros((len(train_words_ner),300)) 

for i in range(len(train_words_ner)):
    words = word_tokenize(train_words_ner[i])
    emb = [model.wv[w] for w in words] 
    X_train_ner[i] = np.mean(emb, axis=0)
    
df_train_ner= pd.DataFrame(X_train_ner)
df_train_ner.fillna(value=0,inplace=True)

X_train_ner = np.array(df_train_ner)
print(X_train_ner.shape)

#======================================================================
le_ner = LabelEncoder()
Y_train_ner = le_ner.fit_transform(train_entity_ner)

Y_train_ner.shape

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(3268, 300)


(3268,)

In [10]:
train_words_norm = data['words'].tolist()
train_entity_norm = data['entity'].tolist()

#======================================================================

X1_train_norm= np.zeros((len(train_words_norm),300)) 
X2_train_norm= np.zeros((len(train_entity_norm ),300)) 

for i in range(len(train_words_norm)):
    words = word_tokenize(train_words_norm[i])
    emb = [model.wv[w] for w in words] 
    X1_train_norm[i] = np.mean(emb, axis=0) 


for i in range(len(train_entity_norm)):
    X2_train_norm[i] = model.wv[train_entity_norm[i]]
    
X_train_norm= np.concatenate((X1_train_norm,X2_train_norm),axis=1)
print(X_train_norm.shape)

df_train_norm= pd.DataFrame(X_train_norm)
df_train_norm.fillna(value=0,inplace=True)
X_train_norm = np.array(df_train_norm)
print(X_train_norm.shape)

#======================================================================

test_norm = data['norm1'].tolist()

#======================================================================

le_norm = LabelEncoder()
Y_train_norm = le_norm.fit_transform(test_norm)

Y_train_norm.shape

(3268, 600)
(3268, 600)


(3268,)

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

model_ner = RandomForestClassifier(n_estimators=30,criterion='entropy',class_weight='balanced')
model_ner.fit(X_train_ner,Y_train_ner)

model_norm = RandomForestClassifier(n_estimators=30,criterion='entropy',class_weight='balanced')
model_norm .fit(X_train_norm ,Y_train_norm )



RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=30, n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [19]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [20]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [21]:
def norm_test_data(df_test,X_test_ner):
    
#     test_words_norm  = df_test['words'].tolist()
    test_entity_norm = df_test['entity'].tolist()
        
    
#     X1_test_norm = np.zeros((len(test_words_norm ),300)) 
    X2_test_norm= np.zeros((len(test_entity_norm ),300)) 
    print(X2_test_norm.shape) 
    
#     for i in range(len(test_words_norm )):
#         words = word_tokenize(test_words_norm [i])
#         emb = [model.wv[w] for w in words] 
#         X1_test_norm [i] = np.mean(emb, axis=0) 

    X1_test_norm  = X_test_ner
        
    for i in range(len(test_entity_norm)):
        X2_train_norm[i] = model.wv[test_entity_norm[i]]
    
    print(X1_test_norm.shape,X2_test_norm.shape)
    X_test_norm  = np.concatenate((X1_test_norm ,X2_test_norm ),axis=1)
    df= pd.DataFrame(X_test_norm )
    df.fillna(value=0,inplace=True)
    X_test_norm  = np.array(df)
    #print(X_test.shape)
    predict_norm = model_norm.predict(X_test_norm )
    norm = le_norm.inverse_transform(predict_norm)
    
    return norm

In [22]:
def preprocess_test_data(df_dat,org1 ):
#     df_dat = df_dat.drop(df_dat[df_dat.entity == 'Paragraph'].index)
    T_index = df_dat['words'].shape[0]
    df_dat = df_dat.drop(df_dat[df_dat.words == 'Title.'].index)
    df_dat['words'] = df_dat['words'].apply(lambda x: clean_data(x))
    test_words = df_dat['words'].tolist()
#     test_entity = df_dat['entity'].tolist()
    
    sent=[]
    b=[]
    ner_words=[]
    df_test = pd.DataFrame(columns = ['text','words','entity','norm'])
    p=1
    X_test_ner =[]
    pattern = 'NP: {<JJ|JJR|JJS>*<NN|NNS|NNP>*|<NN>|<JJ>|<VBP>}'
    for sen in test_words:
        sent = preprocess(sen)
        cp = nltk.RegexpParser(pattern)
        cs = cp.parse(sent)
        for i,w1 in enumerate(cs):
            if type(w1)==nltk.tree.Tree:
                if w1.label()=='NP':
                    X1_test=[]
                    words=''
                    #print(i+3,w1.leaves())
                    
                    for j,w2 in enumerate(w1):
                        #print(j,w2)
                        words=words+w2[0]+' '
                        X1_test.append(model.wv[w2[0]])
                    X_test_ner.append(np.mean(X1_test,axis=0))
                    df_test = df_test.append({'text':f'T{p + T_index}','words':words}, ignore_index = True)
                    print(words,p + T_index)
                    p=p+1
#             else:
#                 b.append(i)
    
    X_test_ner =np.array(X_test_ner)
#     print(X_test_ner.shape,org1 )
    predict_ner = model_ner.predict(X_test_ner)
    ner_entity=le_ner.inverse_transform(predict_ner)
    df_test['entity'] = ner_entity
    print(ner_entity)
    print(X_test_ner.shape,ner_entity.shape)
    print(df_test.head())
    print(org1 )
    
    
    norm = norm_test_data(df_test,X_test_ner)
    df_test['norm'] = norm
    print(norm)
    return df_dat,df_test
    

In [23]:
def create_a2_file(df_test):
    annotation=[]
    text = df_test['text'].tolist()
    norm = df_test['norm'].tolist()
    for i in range(0,len(norm)):
        if 'NCBI' in norm[i]:
            annotation.append(f'N{i+1}\tNCBI_Taxonomy Annotation:{text[i]} Referent:{os.path.splitext(norm[i])[0][14:]}')
        
        elif 'OBT' in norm[i]:
            annotation.append(f'N{i+1}\tOntoBiotope Annotation:{text[i]} Referent:{os.path.splitext(norm[i])[0][12:]}')
            
    sub = pd.DataFrame(annotation)
    
    return sub

In [24]:
def predict_test_data(path):
    
    org_dat = os.listdir(path)
    z=[]
    for i in range(len(org_dat)):
        
        df_dat = pd.DataFrame(columns = ['text','words','entity'])
        org1 = org_dat[i]
        if org1.endswith('.txt'):
            org_path1 = os.path.join(path,org1)
            text1 = pd.read_table(org_path1,header = None)
            z.append(org1)
            for j in range(0,text1.shape[0]):
                df_dat = df_dat.append({'words':text1[0][j]}, ignore_index = True) 
           
            
            df_dat,df_test = preprocess_test_data(df_dat,org1)
            #print(X_test.shape)
            
            sub = create_a2_file(df_test)
            sub.to_csv(f"submission_norm+ner/{os.path.splitext(org1)[0]}.a2",index=False,header=False)


In [25]:
path = 'BioNLP-OST-2019_BB-norm+ner_test'
predict_test_data(path)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  # This is added back by InteractiveShellApp.init_path()


listeria monocytogenes  3
acta  4
protein interacts  5
phosphatidylinositol  6
bisphosphate vitro  7
n terminal region listeria monocytogenes  8
acta  9
protein conjunction host cell factors  10
sufficient actin polymerization  11
bacterial surface  12
previous data  13
acta  14
barbed ends  15
proteins  16
hypothesis actin polymerization experiments presence  17
acta  18
n terminal fragment  19
protein acta  20
protect  21
ends  22
protein contrast polypeptide prevents  23
pip  24
activity  25
protein  26
gel filtration tryptophan fluorescence experiments  27
purified acta  28
n terminal fragment binds  29
pip  30
pip  31
phosphoinositides  32
novels  33
functional domain  34
acta phosphoinositide  35
n terminal region acta  36
conformational changes  37
acta  38
facilitate binding cell components  39
important acta  40
actin polymerization  41
['Microorganism' 'Habitat' 'Phenotype' 'Habitat' 'Habitat' 'Habitat'
 'Habitat' 'Habitat' 'Phenotype' 'Habitat' 'Habitat' 'Habitat' 'Habitat'


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  # This is added back by InteractiveShellApp.init_path()


 37
complex ci  38
addition  39
anionic detergent las phage enzyme  40
streptavidin beads  41
reaction biotin  42
n epsilon aminocaproyl cystamine  43
n glutaryl l ala l ala l p ro phe p diphenyl ester reactions  44
residues  45
part pocket  46
biotin n epsilon aminocaproyl cystamine  47
n glutaryl alpha l lys l la  48
l pro phe p diphenylester  49
select enzymes  50
specific activity substrate  51
lysine p parameters  52
selection instance efficiency maturation mutant  53
libraries  54
['Habitat' 'Habitat' 'Habitat' 'Habitat' 'Phenotype' 'Habitat' 'Phenotype'
 'Habitat' 'Habitat' 'Habitat' 'Habitat' 'Habitat' 'Phenotype' 'Habitat'
 'Habitat' 'Habitat' 'Habitat' 'Habitat' 'Phenotype' 'Microorganism'
 'Microorganism' 'Habitat' 'Habitat' 'Habitat' 'Habitat' 'Habitat'
 'Phenotype' 'Habitat' 'Habitat' 'Habitat' 'Habitat' 'Microorganism'
 'Habitat' 'Phenotype' 'Phenotype' 'Habitat' 'Habitat' 'Phenotype'
 'Habitat' 'Phenotype' 'Habitat' 'Habitat' 'Habitat' 'Habitat' 'Habitat'
 'Habitat' 'Hab

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  # This is added back by InteractiveShellApp.init_path()


...

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  # This is added back by InteractiveShellApp.init_path()


 4
induced expression cobt cbia glycerol fructose  5
balance redox reaction  6
high content  7
fructose supplementation  8
expression cobt vitamin b content soymilk  9
fermented soybean food  10
['Habitat' 'Habitat' 'Habitat' 'Phenotype' 'Habitat' 'Habitat' 'Habitat'
 'Habitat']
(8, 300) (8,)
  text                                            words     entity norm
0   T3                               glycerol fructose     Habitat  NaN
1   T4  enhance production vitamin b glycerol fructose     Habitat  NaN
2   T5  induced expression cobt cbia glycerol fructose     Habitat  NaN
3   T6                          balance redox reaction   Phenotype  NaN
4   T7                                    high content     Habitat  NaN
BB-norm+ner-F-25955289-001.txt
(8, 300)
(8, 300) (8, 300)
['OntoBiotope OBT:000007' 'OntoBiotope OBT:000019'
 'OntoBiotope OBT:000019' 'OntoBiotope OBT:002712' 'NCBI_Taxonomy 1047'
 'OntoBiotope OBT:000007' 'OntoBiotope OBT:000019'
 'OntoBiotope OBT:000019']


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  # This is added back by InteractiveShellApp.init_path()


attention  12
vegetarians  13
soy product  14
due taste  15
easy digestibility  16
association  17
health  18
soy product defect  19
low vitamin content  20
water  21
soluble vitamin b study  22
co fermentation glycerol  23
fructose soy yogurt enhance vitamin b production  24
reuteri serial combination experiments  25
co fermentation  26
enhance production vitamin b g ml supplementations  27
expression cobt cbia  28
balance redox reaction  29
high content  30
fructose supplementation  31
production vitamin b  32
expression cobt bacteria  33
vitamin b  34
content soy yogurt  35
soybean  36
food  37
alternative food  38
vegetarians  39
['Habitat' 'Habitat' 'Habitat' 'Habitat' 'Habitat' 'Habitat' 'Habitat'
 'Habitat' 'Habitat' 'Habitat' 'Habitat' 'Habitat' 'Habitat' 'Habitat'
 'Habitat' 'Habitat' 'Habitat' 'Phenotype' 'Habitat' 'Habitat' 'Habitat'
 'Habitat' 'Habitat' 'Habitat' 'Habitat' 'Habitat' 'Habitat' 'Habitat']
(28, 300) (28,)
  text                words   entity norm
0  T12       

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
pre = model_class.predict(X_test)

In [7]:
data.to_csv("a.a2",index=False)

## cross validation

In [77]:
from sklearn.model_selection import GridSearchCV

In [87]:
parameters = {'criterion':['gini'],'class_weight':['balanced']}

In [88]:
clf = RandomForestClassifier()

In [121]:
rand_clf = GridSearchCV(clf, parameters, cv=3, scoring = 'accuracy')

In [122]:
rand_clf.fit(X_train,Y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'criterion': ['gini'], 'class_weight': ['balanced']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [127]:
rand_clf.score(X_train,Y)

0.9256425948592412

In [26]:
pd.read_csv('BioNLP-OST-2019_BB-norm+ner_train/BB-norm+ner-448557.a2')

,T3 Habitat 61 70 pediatric
0,T4\tHabitat 178 189\trespiratory
1,T5\tHabitat 256 267\trespiratory
2,T6\tHabitat 281 290\tpediatric
3,T7\tHabitat 339 372\tchildren less than 2 year...
4,T8\tHabitat 418 488\tchildren greater than 9 y...
5,T9\tHabitat 469 480\trespiratory
6,T10\tHabitat 490 495\tMales
7,T11\tHabitat 543 550\tfemales
8,T12\tHabitat 573 578\tmales
9,T13\tPhenotype 600 621\tnonbacterial pathogen


In [27]:
pd.read_csv('C:/Users/user/Desktop/BioNLP-OST-2019_BB-norm_test/BB-norm-132046.a2')

,N1 OntoBiotope Annotation:T3 Referent:OBT:000019
0,N2\tNCBI_Taxonomy Annotation:T4 Referent:208964
1,N3\tOntoBiotope Annotation:T5 Referent:OBT:000019
2,N4\tNCBI_Taxonomy Annotation:T6 Referent:1010796
